# Practical training and testing for deep learning

Say that you completed your project and your network has $x$ % as accuracy on the test set. Is there a way to improve upon it? <br>
In this blog post I am going to give some practical training and testing tips that you should try to **boost your network performance**. <br>
Beaware that these tips should be applied **only after you have completed your project**, not during. 

## Learning rate

Learning rate is a key hyperparameter. What is the **problem** with it? The problem is that is hard to find a perfecet learning rate.
**Solution**: the solution consists in using a mixture of high and low learning rates.

## Learning rate schedule: step
The learning rate schedule introduces the idea of starting with a high lerning rate (like 0.1) and divide it by 10 whenever the the training error plateaus. <br>
In practice the learning rate will be dynamic and not fixed and, over the training, it will look like this:
<img src="img/step_decay.jpg">

Below is shown the effect of this dynamic learnig rate when a ResNet network is trained:
<img src="img/lr_effect.jpg">

## Learning rate schedule: warm-up
All schedules we have defined starts with high learning rates. <<br>
However, for very deep networks (eg. ResNet-110 on CIFAR-10) a high learning rate can slow down the convergence at the beginning of the training (eg. the accuracy remain at chance level for several epochs). **This is usually a symptom of poor initialization**: a way to counteract this is to **use a lower learning rate for a few epochs** until accuracy increases. <br>
So the learning rate over the epochs will look something like this:
<img src="img/lr_warmup.jpg">

## Learning rate schedule: one cycle
Another thing to try is the one cycle learning rate schedule. This type of schedule **modifies the learning rate after each mini-batch** (also referred as *itaration*), not after each epoch as the previous types of schedule. <br>
Formally, if we training for a total number of iterations $l$, the learning rate for each iteration $i$ would be:

<img src="img/onecycle.jpg">

where $p$ is the fraction of iteration to increase the learning rate (eg. 0.3). This means that the 30% of the training time I will increase the learning rate. Usually the values used are 0.3 or less.
The one cycle learning rate schedule looks like this:

<img src="img/lr_onecycle.jpg">

One thing to point out is that in the frameworks (eg. TensorFlow, PyTorch) we fix just $lr_{max}$ and the other two ($lr_0$ nd $lr_{min}$) are derived from $lr_{max}$ saying for example that $lr_0$ is smaller 100 times $lr_{max}$, for instance. The same for $lr_{min}$.
Note that, usually, at the end (so $lr_{min}$) we reach a lower value of the learning rate compared to the starting value $lr_0$.

## Test time best practice
Now I am going to outline some best practices to apply at test time. Recall that you should apply these tips at the very end of your project.

### Ensembles

At test time, more often than note, one is use to just take the trained model and run in on the test set and get an accuracy. This is the standard way to test your model. However, we can do better **squeezing out the last drop of juice** from our trained model using **ensembles**. Ensembles follow the following pipeline:

1) train a multiple (ranomly initialized) models on the same dataset <br>
2) run each model over a test image <br>
3) average the results (eg. take average of softmax outputs, then take the argmax) <br>

This usually icreases the overall perfomance by 1-2%. **Why this should be a good idea?** Because if networks have similar error rates, different networks tend to make different mistakes.


### Snapshot Ensembling
The previous method works well. However we have to train a lot of networks and this could take a lot of time. To counteract this issue, **snapshot ensembling** comes to aid.
By using a **cyclic learning rate schedule**, we can simulate $M$ trainings in the time span of one by taking **snapshots of the parameters** reached at the end of each cycle. <br>
We can use a *cyclic cosine learning rate* which will look like this:
<img src="img/cosine_cyclic.jpg">

So, whenever the this cyclic cosine learnig rate is deacreased, we take a snapshots of the parameter of the model and we save this model. The proceed in doing this until the training has finished. At the end we will end up with $M$ models. By ensembling those models at test time, we usually gain better performance that just doing one cycle of training.

### Polyak average

If we do not want to pay the price of the ensemble at test time (because at runtime we would need to run $M$ models), **we would want to get the benefit ensemble without running ensembles**. How to do so? <br>
One thing we can do is **Polyak average** which is basically the idea of updating the parameter in the same way we are doing so far but **then we keep another copy of them**  ($\theta^{(test)}$) where we accumulate every new version of our parameter which we have in a running average fashion. So actually the parameters used at test time will be a running average of the parameter we have seen at training time. **Why this is a good idea?** Because we will see that the loss will be super noisy. It will bumbs a lot between different minibatches. So this Polyak average of the parameters is actually taking the mean of the crazy movement of the loss and on average will give us better performance than one of these checkpoints alone.

$$\theta^{(i+1)} = \theta^{(i)} - lr \nabla_{\theta} L(\theta; D^{(train)})$$

$$\theta^{(test)} = (1-\rho)\theta^{(i+1)} + \rho\theta^{(test)}$$

$\rho$ is an hyperparameter which represents to who give more weight. Being $\rho$ usually very high, we give more weight to the past.

### Stochastic Weight Averaging
A variation of Polyak average is the so called **Stochastic Weight Averaging**.
Its idea is to combine the multiple annealing (eg. the downside curve of the cyclical learning rate). We take only the snapshots when the learning rate in the downside ad small and, instead of doing an ensemble of that, **we average these weights like we did in Polyak average**, but instead of doing it for all the parameters as Polyak does, we **average** only the good ones (the one where the learning rate is already decreased and so it could have reached a good minima or something like a minima). This shown that it generalize well.

The parameters are calulated in this way:
$$\theta^{(test)} = \frac{\theta^{(cycle)} + n_{cycles}\theta^{(test)}} {n_{cycles} + 1}, n_{cycles} += 1$$